# AI MEGADADOS 23-2

**NOME**: Antônio Amaral Egydio Martins

## Parte 1 - ClinPag!

Na parte 1 da prova de MEGADADOS, iremos trabalhar com a base de dados sintética **clinpag**.

### Insper autograding!

Para receber feedback dos exercício na parte 1, iremos utilizar o `insper autograding`. Se não fez este passo, abra o notebook da aula 01 e faça!

**Sugestão**: para aproveitar o `.env`, crie uma pasta para a prova no mesmo local onde costuma deixar as pastas das **aulas**!

### Instalação da base

Execute os scripts `clinpag_001.sql` e `clinpag_002.sql` no MySQL Workbench. Estes scripts criam uma base **clinpag** e inserem alguns dados de exemplo para resolução da prova.

A base pode ser representada pelo seguinte diagrama do model orelacional (diagrama ER):

<img src="img/deer_clinpag.png">

A base da prova simula um cenário de **consultas médicas**. Nela, estão cadastrados **médicos** que atendem diversas **especialidades** em **clínicas**. Assim, os **pacientes** podem agendar **consultas** de determinadas **especialidades** com **médicos** em determinadas **clínicas**.

Obs:
- Cada paciente tem uma cidade onde reside, mas pode se deslocar para ser atendido.
- O médico pode atender mais de uma especialidade em uma clínica.
- O médico pode atender em mais de uma clínica diferente, sendo de diferentes especialidades ou não.

## Como resolver os exercícios?

Crie a base da prova em sua máquina (passo anterior). Utilize o MySQL Workbench ou o conector para testar as queries. Quando estiver bastante certo de que a resposta está correta, faça a submissão para o servidor.

## Import das bibliotecas

Vamos realizar o import das bibliotecas.

In [4]:
import mysql.connector
from functools import partial
import os
import insperautograder.jupyter as ia
from dotenv import load_dotenv

E vamos criar nosso HELPER de conexão com o banco! Perceba que, uma vez configurado o `.env` não precisaremos mais informar usuários, senhas e URLs!

In [5]:
load_dotenv(override=True)

def get_connection_helper():

    def run_db_query(connection, query, args=None):
        with connection.cursor() as cursor:
            print('Executando query:')
            cursor.execute(query, args)
            for result in cursor:
                print(result)

    connection = mysql.connector.connect(
        host=os.getenv('MD_DB_SERVER'),
        user=os.getenv('MD_DB_USERNAME'),
        password=os.getenv('MD_DB_PASSWORD'),
        database='clinpag',
    )
    return connection, partial(run_db_query, connection)


connection, db = get_connection_helper()

### Notas

Para conferir a nota da correção automática da prova, utilize:

In [21]:
ia.grades(task='ai_md_23_2')

|    | Atividade   | Exercício   |   Peso |   Nota |
|---:|:------------|:------------|-------:|-------:|
|  0 | ai_md_23_2  | ex00        |      0 |      0 |
|  1 | ai_md_23_2  | ex01        |      2 |     10 |
|  2 | ai_md_23_2  | ex02        |      3 |     10 |
|  3 | ai_md_23_2  | ex03        |      3 |     10 |
|  4 | ai_md_23_2  | ex04        |      2 |     10 |
|  5 | ai_md_23_2  | ex05        |      3 |     10 |
|  6 | ai_md_23_2  | ex06        |      3 |     10 |

In [24]:
ia.grades(by='TASK', task='ai_md_23_2')

|    | Tarefa     |   Nota |
|---:|:-----------|-------:|
|  0 | ai_md_23_2 |     10 |

**Exercício 0**: Este exercício não vale nota, é apenas uma brincadeira para aquecimento! Se não estiver de bom humor, pule!

**Pergunta**: Por que o Maciel e Márcio terão que utilizar óculos escuros na segunda parte do curso?

In [22]:
resp_00 = "Porque o Tuzin depois das PI's vai BRILHAR"

In [23]:
ia.sender(answer='resp_00', task='ai_md_23_2', question='ex00', answer_type='pyvar')

interactive(children=(Button(description='Enviar ex00', style=ButtonStyle()), Output()), _dom_classes=('widget…

**Obs**: Esta parte da avaliação vale **6 pontos** na nota da prova.
- 50% da nota será pela correção automática do servidor
- 50% da nota será pela correção pelo professor, que levará em consideração, por exemplo:
    - Quão distante sua solução ficou de uma solução correta
    - Se as queries estão formatadas:
        - Padrão de maiúsculas e minúsculas
        - Tabulações e quebras de linhas
    - Se utiliza *alias* nas tabelas, especialmente em queries com múltiplas tabelas
    
A nota retornada pelo servidor estará no intervalo `0.0` a `10.0` e será multiplicada por `0.3` para compor os `50%` da correção automática.

Ainda, considere que os testes não são exaustivos. Sua solução deve funcionar para qualquer conjunto de dados no *schema* da prova. Tentativas de burlar os testes serão penalizadas.

**Exercício 1**: Crie uma query que retorne a quantidade de especialidades que possuem a palavra `"diagnóstico"` em sua descrição.

**Obs**:
- A coluna retornada deve se chamar `qt_diag`.

In [8]:
sql_ex01 = '''
SELECT
    count(id_especialidade) as qt_diag
FROM
    especialidade
WHERE
    descricao LIKE '%diagnóstico%'
'''

db(sql_ex01)

Executando query:
(4,)


In [9]:
ia.sender(answer='sql_ex01', task='ai_md_23_2', question='ex01', answer_type='pyvar')

interactive(children=(Button(description='Enviar ex01', style=ButtonStyle()), Output()), _dom_classes=('widget…

**Exercício 2**: Crie uma query que retorne, para cada médico, em quantas clínicas diferentes ele atende ou já atendeu.

**Obs**:
- Retorne o id, nome e sobrenome de cada médico (nesta ordem).
- A coluna de contagem deve ser a última e se chamar `qt_clin`.
- Retorne em ordem decrescente por `qt_clin`.
- Médicos que não atendem em nenhuma clínica também devem ser retornados.

In [20]:
# Medico -> Medico_atende_clinica (id_medico) -> clinica (id_clinica)

sql_ex02 = '''
SELECT DISTINCT
    medico.id_medico,
    medico.nome,
    medico.sobrenome,
    IFNULL(count(DISTINCT clinica.id_clinica),0) as qt_clin
FROM
    medico
    LEFT JOIN medico_atende_clinica USING (id_medico)
    LEFT JOIN clinica USING (id_clinica)
GROUP BY
    medico.id_medico
ORDER BY
    qt_clin DESC;
'''

db(sql_ex02)

Executando query:
(2, 'Maria', 'Ferreira', 5)
(8, 'Gabriel', 'Gomes', 3)
(1, 'Pedro', 'Silva', 2)
(10, 'Leonardo', 'Morais', 2)
(4, 'José', 'Santos', 1)
(11, 'Laura', 'Pereira', 1)
(13, 'Gustavo', 'Simões', 1)
(15, 'Vinicius', 'Fernandes', 1)
(3, 'Ana', 'Oliveira', 0)
(5, 'Lucas', 'Alves', 0)
(6, 'Luiz', 'Souza', 0)
(7, 'Ricardo', 'Pereira', 0)
(9, 'Matheus', 'Moreira', 0)
(12, 'Isabela', 'Carvalho', 0)
(14, 'Lucas', 'Barbosa', 0)


In [ ]:
ia.sender(answer='sql_ex02', task='ai_md_23_2', question='ex02', answer_type='pyvar')

interactive(children=(Button(description='Enviar ex02', style=ButtonStyle()), Output()), _dom_classes=('widget…

**Exercício 3**: Foi levantada a necessidade de armazenar, para cada médico, em qual universidade e ano ele obteve a primeira graduação em medicina.

Considere o diagrama atualizado:

<img src="img/deer_clinpag_v2.png">

**Modificações**:
- Foi adicionada uma nova tabela `universidade`.
- A tabela `universidade` tem chave estrangeira para a tabela `cidade`.
- A tabela `medico` passa a ter chave estrangeira para a tabela `universidade`, com campo indicando o id da universidade onde obteve a primeira graduação em medicina.

**Atividade**: Construa a **DDL** para criação da tabela `universidade`.

**Obs**:
- Ignore a *constraint* de chave estrangeira para a tabela de `cidade`. Crie o campo `id_cidade` na tabela `universidade` mas não envie a constraint na query submetida ao servidor!
- Caso a tabela exista, ela deve ser removida e recriada.

In [27]:
# Separando em dois comandos para não dar erro!
sql_ex03_cmd1 = """
DROP TABLE IF EXISTS universidade;
"""

sql_ex03_cmd2 = """
CREATE TABLE IF NOT EXISTS universidade (
    id_universidade INT NOT NULL PRIMARY KEY,
    nome VARCHAR(45) NOT NULL,
    ano_fundacao INT,
    id_cidade INT NOT NULL
);
"""

sql_ex03 = f"""
{sql_ex03_cmd1}
{sql_ex03_cmd2}
"""

db(sql_ex03_cmd1)
db(sql_ex03_cmd2)

Executando query:
Executando query:


In [ ]:
ia.sender(answer='sql_ex03', task='ai_md_23_2', question='ex03', answer_type='pyvar')

interactive(children=(Button(description='Enviar ex03', style=ButtonStyle()), Output()), _dom_classes=('widget…

**Exercício 4**: Considerando o *schema* atualizado no exercício anterior:

<img src="img/deer_clinpag_v2.png">

**Atividade**: Crie uma query de **inserção** na tabela `universidade`.

Deve ser inserido um registro onde:

- O `id` da universidade deve ser `100`
- Fundada em `2012`
- Na cidade de `"São Paulo"`
- Com nome de universidade `"Academia de Medicina Muito Estranha"`

In [39]:
sql_ex04 = '''
INSERT INTO
    universidade (id_universidade, nome, ano_fundacao, id_cidade)
VALUES (
    100,
    'Academia de Medicina Muito Estranha',
    '2012',
    1
);

'''

db(sql_ex04)

Executando query:


IntegrityError: 1062 (23000): Duplicate entry '100' for key 'universidade.PRIMARY'

In [33]:
sql_4 = '''
SELECT 
    id_cidade
FROM
    cidade
WHERE
    nome = 'São Paulo'
'''

db(sql_4)

Executando query:
(1,)


In [ ]:
ia.sender(answer='sql_ex04', task='ai_md_23_2', question='ex04', answer_type='pyvar')

interactive(children=(Button(description='Enviar ex04', style=ButtonStyle()), Output()), _dom_classes=('widget…

**Exercício 5**: Crie uma query que, considerando as consultas de cada especialidade atendida pelo médico, retorne (nesta ordem):

- Nome do médico
- Sobrenome do médico
- Nome da especialidade
- Valor médio da consulta desta especialidade com este médico
- Valor mínimo da consulta desta especialidade com este médico
- Valor máximo da consulta desta especialidade com este médico

**Obs**:
- Retorne apenas as especialidades cujo valor médio das consultas seja acima de `300.0`.
- Ordene de forma crescente por (nesta ordem): nome do médico, sobrenome do médico e nome da especialidade.
- As colunas calculadas devem se chamar (nesta ordem):  `valor_medio`, `valor_min` e `valor_max`.

In [11]:
# Medico -> Medico_atende_clinica (id_medico) -> especialidade (id_especialidade) -> consulta (id_especialidade)


# sql_ex05 = '''
# SELECT DISTINCT
#     medico.nome,
#     medico.sobrenome,
#     especialidade.nome,
#     AVG(consulta.valor) as valor_medio,
#     MIN(consulta.valor) as valor_min,
#     MAX(consulta.valor) as valor_max
# FROM
#     medico
#     INNER JOIN medico_atende_clinica USING(id_medico)
#     INNER JOIN especialidade USING(id_especialidade)
#     INNER JOIN consulta USING(id_especialidade)
# GROUP BY
#     especialidade.id_especialidade,
#     medico.id_medico
# HAVING
#     valor_medio > 300.0
# ORDER BY
#     medico.nome ASC, medico.sobrenome ASC, especialidade.nome ASC
# '''


sql_ex05 = '''
SELECT DISTINCT
    medico.nome,
    medico.sobrenome,
    especialidade.nome,
    AVG(consulta.valor) as valor_medio,
    MIN(consulta.valor) as valor_min,
    MAX(consulta.valor) as valor_max
FROM
    especialidade
    INNER JOIN consulta USING(id_especialidade)
    INNER JOIN medico USING(id_medico)
GROUP BY
    especialidade.id_especialidade,
    medico.id_medico
HAVING
    valor_medio > 300.0
ORDER BY
    medico.nome ASC, medico.sobrenome ASC, especialidade.nome ASC
'''

db(sql_ex05)

Executando query:
('José', 'Santos', 'Psiquiatria', Decimal('585.000000'), Decimal('270.00'), Decimal('900.00'))
('Maria', 'Ferreira', 'Oftalmologia', Decimal('941.666667'), Decimal('250.00'), Decimal('1800.00'))
('Pedro', 'Silva', 'Medicina de Emergência', Decimal('312.000000'), Decimal('100.00'), Decimal('625.00'))
('Pedro', 'Silva', 'Pediatria', Decimal('676.666667'), Decimal('150.00'), Decimal('980.00'))
('Pedro', 'Silva', 'Psiquiatria', Decimal('560.000000'), Decimal('150.00'), Decimal('940.00'))


In [8]:
ia.sender(answer='sql_ex05', task='ai_md_23_2', question='ex05', answer_type='pyvar')

interactive(children=(Button(description='Enviar ex05', style=ButtonStyle()), Output()), _dom_classes=('widget…

**Exercício 6**: Sabemos que devido a indisponibilidade de serviços de saúde, muitas pessoas precisam se deslocar para cidades vizinhas, ou até para outros estados para buscar atendimento.

Uma consultoria foi contratada para analisar os dados das consultas e propor soluções para o problema, diminuindo longos deslocamentos.

Entretanto, há um receio em disponibilizar os dados completos dos indivíduos, uma vez que são dados sensíveis.

Assim, você deve criar uma **view** chamada `view_paciente_consulta_outra_cidade` que contenha informações sobre todas as consultas onde o paciente teve que sair de sua cidade para fazer a consulta.

A view deve conter:
- Nome completo do paciente em hash SHA 256 (alias `hash_paciente`).
- Nome da cidade do paciente (alias `cidade_paciente`).
- Nome da cidade da clínica (alias `cidade_clinica`).
- Valor da consulta.

**Obs**:
- Considere que o nome completo é a concatenação do **nome** + **um espaço** + **sobrenome**.
- Ordene por (nesta ordem): `cidade_paciente`, `cidade_clinica`, `hash_paciente`.
- Note que existem campos `id_cidade` para identificar tanto a cidade do paciente (na tabela `paciente`) quanto a cidade da clínica onde ocorre o atendimento (tabela `clinica`).
- Neste exercício você não precisa se preocupar com permissões ou criação de usuários, apenas crie a view!

In [20]:
sql = '''
DROP VIEW IF EXISTS view_paciente_consulta_outra_cidade
'''
db(sql)
sql_ex06 = '''
CREATE VIEW view_paciente_consulta_outra_cidade AS
    SELECT DISTINCT
        SHA2(GROUP_CONCAT(distinct CONCAT (paciente.nome, " ", paciente.sobrenome) SEPARATOR ' '), 256) AS hash_paciente,
        cidade.nome as cidade_paciente,
        cidade_clinica.cid_clinica as cidade_clinica,
        consulta.valor
    FROM
        paciente
        INNER JOIN cidade USING(id_cidade)
        INNER JOIN consulta USING(id_paciente)
        INNER JOIN (
            SELECT
                cidade.nome as cid_clinica,
                consulta.id_consulta as con_id
            FROM
                paciente
                INNER JOIN consulta USING(id_paciente)
                INNER JOIN clinica USING(id_clinica)
                INNER JOIN cidade ON clinica.id_cidade = cidade.id_cidade
        ) AS cidade_clinica ON con_id = consulta.id_consulta
    GROUP BY
        paciente.id_paciente, consulta.id_consulta, cidade_clinica.cid_clinica
    HAVING
        cidade_paciente != cidade_clinica
    ORDER BY
     cidade_paciente, cidade_clinica, hash_paciente, consulta.valor
'''


db(sql_ex06)

Executando query:
Executando query:


In [ ]:
ia.sender(answer='sql_ex06', task='ai_md_23_2', question='ex06', answer_type='pyvar')

interactive(children=(Button(description='Enviar ex06', style=ButtonStyle()), Output()), _dom_classes=('widget…

## Parte 2 - Campeonato!

Por conta do grande aumento do interesse dos alunos em jogos de quebra-cabeças com o objetivo de encaixar blocos, a Atlética resolveu organizar o primeiro campeonato Insper de Montagem de Blocos. Para isso, a Atlética precisa de um sistema para gerenciar as inscrições dos alunos e a pontuação de cada um. Você ficou responsável por desenvolver o banco de dados que será utilizado para armazenar as informações do campeonato.

Em conversas com o cliente, você levantou as seguintes informações sobre o domínio de negócios:

- É necessário manter um cadastro dos alunos do Insper, contendo informações suficientes para identificá-los e também para entrar em contato com eles. Além disso, cada aluno pode escolher um apelido para ser utilizado no campeonato que deve ser único.
- Neste campeonato haverão duas modalidades: Individual e Duplas. Os alunos podem se inscrever nas duas modalidades ao mesmo tempo.
- É necessário saber quando um aluno se inscreveu em uma determinada modalidade e se ele pagou ou não a taxa de inscrição.
- Cada partida é disputada entre dois jogadores ou duas duplas. É necessário manter um cadastro dos jogadores que participaram de cada partida, indicando se eles ganharam ou perderam a partida.

Utilize estas informações para propor soluções adequadas para os próximos exercícios.

**Obs**: Esta parte da avaliação vale **4 pontos** na nota da prova:
- 1.50 para o exercício 7
- 0.75 para o exercício 8
- 0.75 para o exercício 9
- 1.00 para o exercício 10

**Exercício 7**: Desenhe o diagrama do modelo relacional deste problema (pode ser o feito no workbench). Não esqueça de indicar claramente os tipos, chaves primárias, chaves estrangeiras, e a cardinalidade dos relacionamentos.

**Obs:** Salve a imagem do diagrama na pasta `img`. Edite na resposta o caminho para a imagem!

<div class="alert alert-success">
    
Edite na resposta o caminho para a imagem!
    
<img src="img/exemplo.png">

</div>

**Opcional**: caso julgue necessário, utilize o bloco abaixo para justificar decisões tomadas no desenho do diagrama.

<div class="alert alert-success">
    
Nesta imagem, matricula deverá ser chave primária e o campo apelido será único. Multiplos alunos podem realizar multiplas incrições em multiplos campeonatos - a um nível individual, 1 aluno poderá se inscrever em multiplos camponatos, ficando guardado em inscrição sua matricula e o estado de pagamento. Algo importante de ser citado é quanto a inscrição de um aluno em multiplas modalidades, neste caso não haverá uma tabela intermediária entre inscrição e modalidade, mas sim, para 1 ano, serão geradas duas inscrições, uma para cada modalidade.

Quanto ao Time, em 1 time pode-se ter multiplos ou apenas 1 aluno.

Quanto a partida, devem-se ter multiplos times em uma partida.

</div>

**Exercício 8**: Quais entidades você julgou serem necessárias para resolver o problema? Qual o tipo dos relacionamentos entre elas (um-para-muitos, muitos-para-muitos)? Justifique sua resposta.

<div class="alert alert-success">
    
Para resolver o problema é necessário a criação de 4 entidades principais não identificadoras, Alunos, Campeonatos, Time e Partida. Com três entidades intermediárias identificadoras.

No diagrama desenhado, multiplos alunos podem ter inscrições em multiplos campeonatos - visto que pode-se increver em mais de uma modalidade em dado campeonato, cada uma destas inscrições terá a matricula do aluno e um estado se foi pago ou não.

Dos alunos que tiverem o status de pago, multiplos ou apenas 1 podem partipar de um time - que pode ser gerado a partir de duplas ou alunos individuais. Havendo duas relações.

Multiplos times poderão estar em apenas 1 partida - neste caso são dois times para uma partida.

</div>

**Exercício 9**: Em quais colunas você criaria índices? Por que você criaria esses índices?

<div class="alert alert-success">
    
Índices são utilizados de diversas formas, principalmente para melhorar a performance de uma Query em um banco de dados. Para o problema relatado, não há necessidade de criar indices para buscar diretamente alunos, visto que é uma busca que não depende de outras tabelas para ser feita. Porém buscas como inscrições, times e principalmente partidas - precisam de outras informações em conjunto, por guardarem o id de cada relação - como é o exemplo de partida que irá ter o id do time 1 e o id do time 2 - que por sua vez terá os ids dos alunos participantes no time.

Quanto a coluna de Inscrição, não haveria ganhos grotescos em performance, visto que ambas não têm relações de alta complexidade, porém a criação do índice poderá melhorar o tempo de busca - principalmente visto que podem existir multiplos campeonatos em inscrições para dado aluno.

Já a coluna de times, das três relatadas nesta questão é a que apresentaria menor ganho se criado um índice, visto que há apenas o armazenamento da relação de alunos no time, porém um índice poderia ser criado para simplificar a query de busca.

</div>

**Exercício 10**: Dos relacionamentos descritos, quais destes são identificadores e quais são não-identificadores? Por que você tomou essa decisão?

<div class="alert alert-success">

Da situação relatada existem 3 relacionamentos, e 7 tabelas. Iniciando pelo relacionamento Alunos e campeonato, multiplos alunos podem se inscrever em multiplos campeonatos, havendo a necessidade de uma tabela intermediária que armazenada os indicadores/relacionamento de ambas as tabelas, esta tabela irei chamar de inscrições. Sendo assim Alunos e Campeonato são não identificadores e a tabela Inscrições é identificadora.

Partindo agora para a relação entre Alunos e Time. Um time pode ter de 1 a dois alunos, havendo a necessidade da criação de uma tabela para o armazenamento desta relação - tendo em vista que a tabela Time apenas terá informações essenciais para sua criação, e não deverá armazenar os alunos diretamente. Send assin a tabela de Alunos e Time são não identificadoras, e a tabela Tem (que intermedia ambos) é indentificadora.

A última relação ocorre entre Time e Partida, em que uma partida pode e deverá ter múltiplos times, havendo a necessidade de se criar uma tabela para armazenar este relacionamento, chamada de Tem. Nesta situação, a tabela Partida irá armazenar apenas as informações daquela partida, sendo assim Time e Partida são não identificadoras, enquanto Tem é identificadora.

</div>

## Entrega!

É hora de entregar. Faça um **zip** do notebook + imagens chamado `ai_megadados.zip` e submeta no Blackboard!

**Obs**:
- `zip`, não use `rar`!
- Não precisa entregar os scripts fornecidos pelo professor (pasta scripts)